# Data Cleaning and Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.getcwd()

'C:\\Users\\zhika\\Desktop\\NCAA_Data\\Stage2DataFiles'

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'ConferenceTourneyGames.csv',
 'GameCities.csv',
 'NCAATourneyCompactResults.csv',
 'NCAATourneyDetailedResults.csv',
 'NCAATourneySeedRoundSlots.csv',
 'NCAATourneySeeds.csv',
 'NCAATourneySlots.csv',
 'RegularSeasonCompactResults.csv',
 'RegularSeasonDetailedResults.csv',
 'Seasons.csv',
 'SecondaryTourneyCompactResults.csv',
 'SecondaryTourneyTeams.csv',
 'TeamCoaches.csv',
 'TeamConferences.csv',
 'Teams.csv',
 'TeamSpellings.csv',
 'Untitled.ipynb']

In [6]:
seasons = pd.read_csv('Seasons.csv')
t_compact = pd.read_csv('NCAATourneyCompactResults.csv')
r_compact = pd.read_csv('RegularSeasonCompactResults.csv')
seeds = pd.read_csv('NCAATourneySeeds.csv')

In [9]:
t_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [10]:
r_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [11]:
seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


#### Check if following datasets have any missing values

In [15]:
missing_values_t_compact = t_compact.isnull().sum()
missing_values_t_compact

Season     0
DayNum     0
WTeamID    0
WScore     0
LTeamID    0
LScore     0
WLoc       0
NumOT      0
dtype: int64

In [16]:
missing_values_r_compact = r_compact.isnull().sum()
missing_values_r_compact

Season     0
DayNum     0
WTeamID    0
WScore     0
LTeamID    0
LScore     0
WLoc       0
NumOT      0
dtype: int64

In [17]:
missing_values_seeds = seeds.isnull().sum()
missing_values_seeds

Season    0
Seed      0
TeamID    0
dtype: int64

#### Remove letter from Seed column

In [18]:
# Function that removes letter from seeds

def seed_num(seed):
    seed_number = int(seed[1:3])
    return seed_number
seeds['Seed_Num'] = seeds.Seed.apply(seed_num)
seeds.drop(['Seed'], inplace = True, axis = 1)

seeds.head()

,Season,TeamID,Seed_Num
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


#### Get the seed difference between teams in tournament

In [25]:
winners = seeds.rename(columns = {'TeamID': 'WTeamID', 'Seed_Num':'WSeed'})
losers = seeds.rename(columns = {'TeamID': 'LTeamID', 'Seed_Num': 'LSeed'})
winners_t = pd.merge(left=t_compact, right=winners, how = 'left', on =['Season', 'WTeamID'])
new_data = pd.merge(left= winners_t, right=losers, on =['Season', 'LTeamID'])

new_data['Seed_Difference'] = new_data.LSeed - new_data.WSeed
new_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,LSeed,Seed_Difference
0,1985,136,1116,63,1234,54,N,0,9,8,-1
1,1985,136,1120,59,1345,58,N,0,11,6,-5
2,1985,136,1207,68,1250,43,N,0,1,16,15
3,1985,136,1229,58,1425,55,N,0,9,8,-1
4,1985,136,1242,49,1325,38,N,0,3,14,11


In [26]:
# Drop unnecessary columns
new_data.drop(['DayNum', 'WScore', 'LScore', 'WLoc',
              'NumOT'], inplace =True, axis = 1)

In [27]:
new_data.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,Seed_Difference
0,1985,1116,1234,9,8,-1
1,1985,1120,1345,11,6,-5
2,1985,1207,1250,1,16,15
3,1985,1229,1425,9,8,-1
4,1985,1242,1325,3,14,11


In [28]:
new_data.tail()

,Season,WTeamID,LTeamID,WSeed,LSeed,Seed_Difference
2179,2018,1242,1181,1,2,1
2180,2018,1437,1403,1,3,2
2181,2018,1276,1260,3,11,8
2182,2018,1437,1242,1,1,0
2183,2018,1437,1276,1,3,2


#### Get Regular Season Games won per team

In [32]:
r_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [35]:
#r_compact.loc[(r_compact['Season'] == 1985) & (r_compact['WTeamID'] == 1116)]
# Drop unnecessary columns
r_compact.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace = True, axis = 1)

In [36]:
r_compact.head()

,Season,WTeamID,LTeamID
0,1985,1228,1328
1,1985,1106,1354
2,1985,1112,1223
3,1985,1165,1432
4,1985,1192,1447


In [45]:
teams = pd.read_csv('Teams.csv')

In [46]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2019
1,1102,Air Force,1985,2019
2,1103,Akron,1985,2019
3,1104,Alabama,1985,2019
4,1105,Alabama A&M,2000,2019


In [47]:
teams.drop(['FirstD1Season', 'LastD1Season'], inplace = True, axis = 1)

In [48]:
teams.head()

,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [51]:
# Include team names in the dataset

winnersID = teams.rename(columns = {'TeamID': 'WTeamID'})
losersID = teams.rename(columns = {'TeamID': 'LTeamID'})
winners_t_ID = pd.merge(left=new_data, right=winnersID, how = 'left', on =['WTeamID'])
data = pd.merge(left= winners_t_ID, right=losersID, on =['LTeamID'])

data.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,Seed_Difference,TeamName_x,TeamName_y
0,1985,1116,1234,9,8,-1,Arkansas,Iowa
1,1986,1301,1234,6,11,5,NC State,Iowa
2,1987,1424,1234,1,2,1,UNLV,Iowa
3,1988,1112,1234,1,5,4,Arizona,Iowa
4,1989,1301,1234,5,4,-1,NC State,Iowa


In [52]:
data = data.rename(columns = {'TeamName_x' : 'WTeamName', 'TeamName_y': 'LTeamName'})

In [53]:
data.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,Seed_Difference,WTeamName,LTeamName
0,1985,1116,1234,9,8,-1,Arkansas,Iowa
1,1986,1301,1234,6,11,5,NC State,Iowa
2,1987,1424,1234,1,2,1,UNLV,Iowa
3,1988,1112,1234,1,5,4,Arizona,Iowa
4,1989,1301,1234,5,4,-1,NC State,Iowa


#### Get the results of each game

In [85]:
df_wins = pd.DataFrame()
df_wins[['Season', 'WTeamID', 'LTeamID', 'WSeed', 'LSeed', 
         'WTeamName', 'LTeamName']] = data[['Season', 
        'WTeamID', 'LTeamID', 'WSeed', 'LSeed', 'WTeamName', 'LTeamName']]
df_wins['Seed_Difference'] = data['Seed_Difference']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses[['Season', 'WTeamID', 'LTeamID', 'WSeed', 'LSeed', 
         'WTeamName', 'LTeamName']] = data[['Season', 
        'WTeamID', 'LTeamID', 'WSeed', 'LSeed', 'WTeamName', 'LTeamName']]
df_losses['Seed_Difference'] = -data['Seed_Difference']
df_losses['Result'] = 0

df_results = pd.concat((df_wins, df_losses))
df_results.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,WTeamName,LTeamName,Seed_Difference,Result
0,1985,1116,1234,9,8,arkansas,iowa,-1,1
1,1986,1301,1234,6,11,nc state,iowa,5,1
2,1987,1424,1234,1,2,unlv,iowa,1,1
3,1988,1112,1234,1,5,arizona,iowa,4,1
4,1989,1301,1234,5,4,nc state,iowa,-1,1


#### Importing Draft Dataset

In [97]:
draft_results = pd.read_pickle(r"C:\Users\zhika\Desktop\NCAA_Data\Stage2DataFiles\draft_results.pkl")

In [98]:
draft_results.head()

,College,Season,num_drafted,best_college_player
1,georgetown,1985.0,2.0,1.0
2,oklahoma,1985.0,1.0,0.0
3,creighton,1985.0,1.0,0.0
4,wichita state,1985.0,1.0,0.0
5,smu,1985.0,1.0,0.0


In [56]:
df_results['WTeamName'] = df_results['WTeamName'].str.lower()

In [57]:
df_results['LTeamName'] = df_results['LTeamName'].str.lower()

#### Making sure all the team names match for both datasets

In [75]:
teams['TeamName'] = teams['TeamName'].str.lower()
teams_names = teams['TeamName'].unique()
teams_names.sort()
teams_names

array(['abilene chr', 'air force', 'akron', 'alabama', 'alabama a&m',
       'alabama st', 'albany ny', 'alcorn st', 'alliant intl',
       'american univ', 'appalachian st', 'arizona', 'arizona st',
       'ark little rock', 'ark pine bluff', 'arkansas', 'arkansas st',
       'armstrong st', 'army', 'auburn', 'augusta', 'austin peay',
       'ball st', 'baylor', 'belmont', 'bethune-cookman', 'binghamton',
       'birmingham so', 'boise st', 'boston college', 'boston univ',
       'bowling green', 'bradley', 'brooklyn', 'brown', 'bryant',
       'bucknell', 'buffalo', 'butler', 'byu', 'c michigan',
       'cal baptist', 'cal poly slo', 'california', 'campbell',
       'canisius', 'cent arkansas', 'centenary', 'central conn',
       'charleston so', 'charlotte', 'chattanooga', 'chicago st',
       'cincinnati', 'citadel', 'clemson', 'cleveland st', 'coastal car',
       'col charleston', 'colgate', 'colorado', 'colorado st', 'columbia',
       'connecticut', 'coppin st', 'cornell', 'cre

In [82]:
draft_results.sort()
draft_results

array(['alabama', 'alcorn state university',
       'american international college', 'arizona', 'arizona state',
       'arkansas', 'auburn', 'auburn university at montgomery',
       'augsburg college', 'austin peay state university',
       'ball state university', 'barton county community college',
       'baylor', 'boise state', 'boston college', 'bowling green',
       'bradley', 'bucknell', 'butler', 'butler county community college',
       'byu', 'cal', 'cal state long beach',
       'california state university, bakersfield',
       'california state university, fullerton', 'canisius college',
       'central connecticut state university', 'central florida',
       'central michigan university', 'central state university',
       'chattanooga', 'christopher newport university', 'cincinnati',
       'clemson', 'cleveland state university', 'colgate university',
       'college of charleston', 'colorado', 'colorado state', 'creighton',
       'dartmouth college', 'davidson', 'd

In [99]:
# draft_results["College"]= draft_results["College"].replace("alcorn state university", "alcorn st") 
draft_results = draft_results.replace(to_replace = ["american international college", "arizona state", "austin peay state university", "ball state university", "boise state",
"cal", "canisius college", "central michigan university", "colgate university", "cleveland state university", "colorado state", "central connecticut state university",
"college of charleston", "california state university, bakersfield", "fullerton", "dartmouth college", "detroit mercy", "east carolina university",
"eastern illinois university", "eastern michigan university", "eastern washington university", "florida state", "fresno state", "george mason university", 
"george washington", "georgia state university", "georgia southern university", "iona college", "iowa state", "jackson state university", 
"jacksonville university", "kansas state", "long island university", "loyola chicago", "loyola marymount university", "loyola college in maryland",
"manhattan college", "marist college", "miami (fl)", "miami university", "michigan state", "mississippi state", "missouri state", 
"mcneese state university", "mercer university", "morehead state university", "murray state", "new mexico state", "niagara university",
"northeastern university", "northern arizona university", "northwestern oklahoma state university", "ohio state", "ohio university",
"okaloosa-walton community college", "oklahoma state", "oregon state", "penn state", "pitt", "rice university", "rider university",
"rutgers university", "south dakota state university", "southern illinois", "san diego state", "san jose state", "santa clara university",
"st. john's", "saint francis university", "saint joseph's", "southern university and a&m college", "st. bonaventure", 
"saint louis university", "saint mary's", "tennessee state", "tennessee technological university", "texas state university",
"university of california, irvine", "wright state university", "wichita state", "weber state", "washington state", "university of pennsylvania",
"western carolina university", "west virginia state university", "uconn", "university of new orleans", "university of illinois at chicago", 
"umass", "university of hartford", "central florida", "unc", "university of montana", "university of south alabama", "university of south florida",
"university of north dakota", "university of richmond", "university of idaho", "virginia commonwealth", "university of the pacific", 
"university of texas at san antonio", "university of tennessee at martin", "university of southern mississippi", "university of evansville",
"university of central arkansas", "university of california, santa barbara", "united states naval academy", "unc charlotte", "norfolk state",
"middle tennessee state university", "little rock", "cal state long beach"], 
                                                    value = ["american univ", "arizona st", "austin peay", "ball st", "boise st", "california", "canisius", "c michigan", "colgate", "cleveland st",
"colorado st", "central conn", "col charleston", "cs bakersfield, california state university", "cs fullerton", "dartmouth", "detroit", "east carolina",
"e illinois", "e michigan", "e washington", "florida st", "fresno st", "george mason", "g washington", "georgia st", "ga southern", "iona",
"iowa st", "jackson st", "jacksonville", "kansas st", "long island", "loyola-chicago", "loy marymount", "loyola md", "manhattan", "marist",
"miami fl", "miami oh", "michigan st", "mississippi st", "missouri st", "mcneese st", "mercer", "morehead st", "murray st", "new mexico st",
"niagara", "northeastern", "northern arizona", "northwestern la", "ohio st", "ohio", "okla city", "oklahoma st", "oregon st", "penn st", "pittsburgh",
"rice", "rider", "rutgers", "s dakota st", "s illinois", "san diego st", "san jose st", "santa clara", "st john's", "st francis pa", "st joseph's pa",
"southern univ", "st bonaventure", "st louis", "st mary's ca", "tennessee st", "tennessee tech", "texas st", "uc irvine", "wright st", "wichita st",
"weber st", "washington st", "penn", "w carolina", "west virginia", "connecticut", "new orleans", "il chicago", "massachusetts", "hartford",
"ucf", "north carolina", "montana", "south alabama", "south florida", "north dakota", "richmond", "idaho", "va commonwealth", "pacific",
"ut san antonio", "tn martin", "southern miss", "evansville", "cent arkansas", "santa barbara", "navy", "charlotte", "norfolk state",
"mtsu", "ark little rock", "long beach st"])

In [101]:
draft_results.head()

,College,Season,num_drafted,best_college_player
1,georgetown,1985.0,2.0,1.0
2,oklahoma,1985.0,1.0,0.0
3,creighton,1985.0,1.0,0.0
4,wichita st,1985.0,1.0,0.0
5,smu,1985.0,1.0,0.0


In [111]:
winners_draft_res = draft_results.rename(columns = {'College': 'WTeamName'})
losers_draft_res = draft_results.rename(columns = {'College': 'LTeamName'})
winners_draft = pd.merge(left=df_results, right=winners_draft_res, how = 'left', on =['Season', 'WTeamName'])
new_data = pd.merge(left= winners_draft, right=losers_draft_res, on =['Season', 'LTeamName'])

In [108]:
new_data.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,WTeamName,LTeamName,Seed_Difference,Result,num_drafted_x,best_college_player_x,num_drafted_y,best_college_player_y
0,1985,1116,1234,9,8,arkansas,iowa,-1,1,1.0,0.0,1.0,0.0
1,1985,1116,1234,9,8,arkansas,iowa,1,0,1.0,0.0,1.0,0.0
2,1987,1424,1234,1,2,unlv,iowa,1,1,1.0,0.0,2.0,0.0
3,1987,1424,1234,1,2,unlv,iowa,1,1,1.0,0.0,2.0,0.0
4,1987,1424,1234,1,2,unlv,iowa,-1,0,1.0,0.0,2.0,0.0


In [112]:
new_data = new_data.rename(columns = {'num_drafted_x': 'drafted_from_win', 'best_college_player_x': 'best_player_on_win',
                                     'num_drafted_y': 'drafted_from_lose'})

In [113]:
new_data.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,WTeamName,LTeamName,Seed_Difference,Result,drafted_from_win,best_player_on_win,drafted_from_lose,best_college_player_y
0,1985,1116,1234,9,8,arkansas,iowa,-1,1,1.0,0.0,1.0,0.0
1,1985,1116,1234,9,8,arkansas,iowa,1,0,1.0,0.0,1.0,0.0
2,1987,1424,1234,1,2,unlv,iowa,1,1,1.0,0.0,2.0,0.0
3,1987,1424,1234,1,2,unlv,iowa,1,1,1.0,0.0,2.0,0.0
4,1987,1424,1234,1,2,unlv,iowa,-1,0,1.0,0.0,2.0,0.0


In [114]:
new_data.drop(['Season', 'WTeamID', 'LTeamID', 'best_college_player_y'], inplace = True, axis = 1)

In [117]:
new_data['Seed_Difference'] = new_data['LSeed'] - new_data['WSeed']

In [119]:
new_data.head()

,WSeed,LSeed,WTeamName,LTeamName,Seed_Difference,Result,drafted_from_win,best_player_on_win,drafted_from_lose
0,9,8,arkansas,iowa,-1,1,1.0,0.0,1.0
1,9,8,arkansas,iowa,-1,0,1.0,0.0,1.0
2,1,2,unlv,iowa,1,1,1.0,0.0,2.0
3,1,2,unlv,iowa,1,1,1.0,0.0,2.0
4,1,2,unlv,iowa,1,0,1.0,0.0,2.0


In [120]:
# Rearrange the columns in dataframe
new_data = new_data[['WTeamName', 'LTeamName', 'WSeed','LSeed','Seed_Difference',
                    'drafted_from_win', 'drafted_from_lose', 'best_player_on_win',
                    'Result']]

In [121]:
new_data.head()

,WTeamName,LTeamName,WSeed,LSeed,Seed_Difference,drafted_from_win,drafted_from_lose,best_player_on_win,Result
0,arkansas,iowa,9,8,-1,1.0,1.0,0.0,1
1,arkansas,iowa,9,8,-1,1.0,1.0,0.0,0
2,unlv,iowa,1,2,1,1.0,2.0,0.0,1
3,unlv,iowa,1,2,1,1.0,2.0,0.0,1
4,unlv,iowa,1,2,1,1.0,2.0,0.0,0


In [122]:
new_data.to_pickle('train_data.pkl')